In [ ]:

# ⚽ Sistema de Predicción de Fútbol con IA

# ✅ Cargar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# ✅ Crear datos de ejemplo
data = {
    'equipo_local': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'C', 'A'],
    'equipo_visitante': ['B', 'C', 'C', 'A', 'A', 'C', 'B', 'A', 'A', 'B'],
    'goles_local': [1, 2, 3, 0, 1, 2, 0, 3, 1, 2],
    'goles_visitante': [0, 1, 1, 2, 2, 1, 3, 1, 1, 0]
}
df = pd.DataFrame(data)

# ✅ Crear columna de resultado (1 si gana local, 0 si no)
df['resultado'] = (df['goles_local'] > df['goles_visitante']).astype(int)

# ✅ Convertir equipos en variables dummy
df_dummies = pd.get_dummies(df[['equipo_local', 'equipo_visitante']])
X = pd.concat([df_dummies], axis=1)
y = df['resultado']

# ✅ Separar datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Entrenar modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# ✅ Hacer predicciones
y_pred = model.predict(X_test)

# ✅ Evaluar modelo
acc = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {acc:.2f}")

# ✅ Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()

# ✅ Predicción personalizada
nuevo_partido = pd.DataFrame({
    'equipo_local_A': [1],
    'equipo_local_B': [0],
    'equipo_local_C': [0],
    'equipo_visitante_A': [0],
    'equipo_visitante_B': [1],
    'equipo_visitante_C': [0],
})
prediccion = model.predict(nuevo_partido)
print("Predicción (1 = gana local, 0 = pierde o empata):", prediccion[0])


In [ ]:

# 📂 Cargar archivo Excel o CSV para predicción masiva

from google.colab import files
import io

# Subir archivo
uploaded = files.upload()

# Leer archivo Excel o CSV
for fname in uploaded.keys():
    if fname.endswith('.csv'):
        df_input = pd.read_csv(fname)
    elif fname.endswith('.xlsx'):
        df_input = pd.read_excel(fname)
    else:
        print("Formato no compatible:", fname)
        df_input = None

# Predecir si el archivo es válido
if df_input is not None:
    predicciones = model.predict(df_input)
    df_input['Predicción'] = ['🏠 Gana local' if p == 1 else '🤝 Empate o gana visitante' for p in predicciones]
    display(df_input)


In [ ]:

# 💾 Exportar predicciones a Excel

from datetime import datetime

if 'df_input' in locals():
    nombre_archivo = f"predicciones_futbol_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    df_input.to_excel(nombre_archivo, index=False)
    print(f"✅ Archivo exportado: {nombre_archivo}")
else:
    print("⚠️ No se encontraron predicciones recientes para exportar.")


In [ ]:

# 🖱️ Interfaz visual mejorada con widgets para historial

import ipywidgets as widgets
from IPython.display import display, clear_output

# Menús desplegables
equipos = ['A', 'B', 'C']
equipo_local = widgets.Dropdown(options=equipos, description='Local:')
equipo_visitante = widgets.Dropdown(options=equipos, description='Visitante:')
boton = widgets.Button(description='Predecir Resultado', button_style='info')
salida = widgets.Output()

# Historial de predicciones
historial = []

def hacer_prediccion(b):
    with salida:
        clear_output()
        datos = {
            'equipo_local_A': [1 if equipo_local.value == 'A' else 0],
            'equipo_local_B': [1 if equipo_local.value == 'B' else 0],
            'equipo_local_C': [1 if equipo_local.value == 'C' else 0],
            'equipo_visitante_A': [1 if equipo_visitante.value == 'A' else 0],
            'equipo_visitante_B': [1 if equipo_visitante.value == 'B' else 0],
            'equipo_visitante_C': [1 if equipo_visitante.value == 'C' else 0],
        }
        nuevo = pd.DataFrame(datos)
        pred = model.predict(nuevo)[0]
        resultado = "🏠 Gana local" if pred == 1 else "🤝 Empate o gana visitante"
        print(f"📊 {equipo_local.value} vs {equipo_visitante.value}: {resultado}")
        historial.append({'Local': equipo_local.value, 'Visitante': equipo_visitante.value, 'Resultado': resultado})

# Botón vinculado
boton.on_click(hacer_prediccion)

# Mostrar interfaz
display(widgets.VBox([equipo_local, equipo_visitante, boton, salida]))


In [ ]:

# 📝 Guardar historial de predicciones en Excel

from openpyxl import Workbook
import os

if historial:
    df_historial = pd.DataFrame(historial)
    nombre_historial = "historial_predicciones.xlsx"
    df_historial.to_excel(nombre_historial, index=False)
    print(f"✅ Historial guardado en: {nombre_historial}")
else:
    print("⚠️ Aún no hay historial para guardar.")
